# Atelier sur la génétique statistique et l'épidémiologie génétique STAGE-Québec
## Thème 2 - Phénotypes moléculaires en épidémiologie génétique

Par Marc-André Legault (Université de Montréal) et Qihuang Zhang (Université McGill)

**31 juillet - 1er août 2025**

### Introduction

Dans le carnet précédent, nous avons réalisé une analyse TWAS (Transcriptome-Wide Association Study) du diabète de type 2 pour 7 tissus GTEx en utilisant les modèles de prédiction de l'expression génique de PredictDB et la méthode S-PrediXcan. 

Dans ce carnet, nous examinerons ces résultats pour en comprendre les implications biologiques. L'objectif principal consiste à interpréter et visualiser les associations significatives entre l'expression génique prédite et le diabète de type 2 dans différents tissus.

**Première étape : Résumé des gènes significatifs par tissu**

Nous examinerons le nombre de gènes significatifs identifiés dans chaque tissu. Pour ce faire, nous devons d'abord calculer les seuils de valeur P appropriés selon différentes approches de correction pour tests multiples :

| Tissu                | N gènes testés | Bonferroni global | Bonferroni par tissu |
| :------------------: | -------------- | ----------------- | -------------------- |
| Tissu adipeux sous-cutané |            |                   |                      |
| ...                  |                |                   |                      |

In [ ]:
library(ggplot2)

In [ ]:
tissues <- c(
    "en_Adipose_Subcutaneous",
    "en_Artery_Coronary",
    "en_Brain_Cortex",
    "en_Liver",
    "en_Muscle_Skeletal",
    "en_Pancreas",
    "en_Whole_Blood"
)

df <- data.frame()
for (tissue in tissues) {
    cur <- read.csv(paste0("/workshop/local/results/twas_", tissue, ".csv"))
    df <- rbind(df, data.frame(tissue = tissue, n_tested_genes = nrow(cur)))
}

df$global_bonf <- 0.05 / sum(df$n_tested_genes)
df$tissue_bonf <- 0.05 / df$n_tested_genes
df

**Analyse des seuils de signification**

Les résultats montrent qu'une correction de Bonferroni complète tenant compte de tous les tests effectués à travers l'ensemble des tissus demeure moins stricte que le seuil de signification génome-entier traditionnel ($5 \times 10^{-8}$). 

Le nombre de gènes testés varie considérablement entre les tissus : de 3 765 gènes pour le foie jusqu'à 8 626 gènes pour le tissu adipeux sous-cutané. Cette variation explique les différences observées dans les seuils de Bonferroni spécifiques aux tissus.

Nous examinerons maintenant le nombre de gènes significatifs selon différentes approches de correction statistique. Nous inclurons également la correction du taux de fausses découvertes (FDR) de Benjamini-Hochberg, une méthode moins conservatrice que la correction de Bonferroni.

<div class="alert alert-info">
<strong>Question de réflexion :</strong> Pourquoi le nombre de gènes testés varie-t-il selon les tissus ? Quels facteurs biologiques et techniques peuvent expliquer ces différences ?
</div>

In [ ]:
n_sig <- data.frame()
sig_genes_data <- data.frame()
sig_genes <- list()

for (tissue in tissues) {
    cur <- read.csv(paste0("/workshop/local/results/twas_", tissue, ".csv"))

    sig_global_bonf <- cur$pvalue <= 1.174205e-6
    sig_tissue_bonf <- cur$pvalue <= 0.05 / nrow(cur)
    q <- p.adjust(cur$pvalue, method = "BH")
    sig_fdr <- q <= 0.05
    
    sig_genes_data_cur <- cur[sig_fdr, c("gene_name", "pvalue", "effect_size", "n_snps_in_model", "n_snps_used", "pred_perf_pval")]
    sig_genes_data_cur$tissue <- tissue
    sig_genes_data <- rbind(sig_genes_data, sig_genes_data_cur)
    
    sig_genes[[tissue]] <- list(
        sig_global_bonf = cur[sig_global_bonf, "gene_name"],
        sig_tissue_bonf = cur[sig_tissue_bonf, "gene_name"],
        sig_fdr = cur[sig_fdr, "gene_name"]
    )
    
    n_sig <- rbind(n_sig, data.frame(
        tissue = tissue,
        sig_global_bonf = sum(sig_global_bonf),
        sig_tissue_bonf = sum(sig_tissue_bonf),
        sig_fdr = sum(sig_fdr)
    ))
}
n_sig

<div class="alert alert-info">
<strong>Analyse comparative des méthodes de correction :</strong>
<ul>
<li>Quelle correction est la plus stricte ? La moins stricte ?</li>
<li>Quelle approche est la plus appropriée dans le contexte d'une analyse TWAS ?</li>
<li>Quelles sont les hypothèses sous-jacentes de chacune de ces méthodes de correction pour tests multiples ?</li>
<li>Comment le choix de la méthode influence-t-il l'interprétation biologique des résultats ?</li>
</ul>
</div>

In [ ]:
cat("Gènes significatifs à FDR <= 5% à travers les tissus:\n\n")
for (tissue in tissues) {
    cat(paste0(
        "\t'", tissue, "': ",
        paste0(sig_genes[[tissue]]$sig_fdr, collapse = ", "),
        "\n"
    ))
}

**Identification des gènes candidats**

<div class="alert alert-info">
<strong>Observations importantes :</strong>
<p>L'expression génétiquement prédite d'<em>AP3S2</em> est associée au diabète de type 2 dans <strong>tous</strong> les tissus considérés, suggérant un rôle biologique fondamental de ce gène dans la pathogenèse du diabète.</p>

<p>D'autres gènes présentent des effets spécifiques aux tissus. Par exemple, l'expression de <em>FTO</em> n'est associée au diabète de type 2 que dans le muscle squelettique.</p>

<p><strong>Question d'interprétation :</strong> Qu'est-ce qui pourrait expliquer cette spécificité tissulaire pour <em>FTO</em> ? Considérez les aspects suivants :</p>
<ul>
<li>La spécificité biologique de l'effet du gène selon le tissu</li>
<li>L'héritabilité variable de l'expression génique entre les tissus</li>
<li>Les différences de puissance statistique pour détecter les eQTL selon les tissus</li>
<li>La taille des échantillons disponibles pour chaque tissu dans GTEx</li>
</ul>
</div>

**Visualisation des résultats TWAS**

Pour analyser les patterns d'association et examiner la directionnalité des effets, nous créerons un diagramme en points qui résume visuellement les résultats significatifs :

In [ ]:
source("/workshop/utilities/gene_tissue_dotplot.R")
options(repr.plot.width = 10, repr.plot.height = 5)
options(repr.plot.res = 200)
plot_gene_tissue_dots_advanced(sig_genes_data)

**Interprétation thérapeutique des résultats**

<div class="alert alert-info">
<strong>Analyse des implications thérapeutiques :</strong>
<p>En supposant que les associations observées reflètent des relations causales, quel gène représenterait la cible thérapeutique la plus prometteuse pour prévenir le diabète de type 2 ? Dans quel(s) tissu(s) devrait-on cibler cette intervention ?</p>

<p><strong>Critères d'évaluation :</strong></p>
<ul>
<li>Les gènes présentant des associations <strong>positives</strong> et <strong>robustes</strong> (représentées en rouge dans le graphique)</li>
<li>La spécificité tissulaire des effets</li>
<li>La faisabilité biologique de l'inhibition génique dans différents tissus</li>
</ul>

<p><em>Hypothèse de travail : WFS1 dans le tissu adipeux ou ANK1 dans le muscle squelettique pourraient constituer des candidats thérapeutiques intéressants.</em></p>
</div>

**Analyse de région génomique : Focus sur _WFS1_**

Pour approfondir notre compréhension des mécanismes sous-jacents, nous examinerons en détail la région génomique du gène _WFS1_. Nous extrairons les statistiques sommaires GWAS correspondantes et créerons un graphique de région qui mettra en évidence les variants génétiques inclus dans le modèle prédictif :

In [ ]:
# Nous définissons la région WFS1 en utilisant les données d'Ensembl.
# Voir : https://useast.ensembl.org/Homo_sapiens/Gene/Summary?db=core;g=ENSG00000109501;r=4:6269849-6303265
# Nous avons ajouté un espacement de +/- 100kb.
wfs1_region <- list(
    chrom = "chr4",
    start = 6269849 - 100000,
    end = 6303265 + 100000
)

get_predictdb_and_gwas_data <- function(region, gene_name, tissue) {
    # Nous lisons les statistiques sommaires GWAS, mais gardons seulement l'information pour
    # les variants dans la région d'intérêt.
    gwas <- read.csv(
        paste0(
            "/workshop/data/PrediXcan/gwas_harmonized/harmonized_",
            tissue,
            ".tsv.gz"
        ), sep="\t"
    )
    gwas_region <- gwas[
        (gwas$chromosome == region$chrom) &
        (region$start <= gwas$base_pair_location) &
        (gwas$base_pair_location <= region$end),
    ]
    rm(gwas)

    # Nous extrairons le rsid et les poids des variants inclus dans le
    # modèle prédictif de l'expression génique.
    db_filename = paste0("/workshop/data/PrediXcan/predictdb/", tissue, ".db")
        
    model <- system(paste0(
        "sqlite3 ",
        db_filename,
        " \"select rsid, weight from weights w, extra e where e.gene=w.gene and e.genename='",
        gene_name, "';\""    ), intern=T)

    if (length(model) == 0) {
        stop("Impossible de trouver les données PredictDB pour le gène.")
    }

    model <- data.frame(do.call(rbind, strsplit(model, "\\|")))
    colnames(model) <- c("rsid", "weight")
    model$weight <- as.numeric(model$weight)

    # Nous créons des variables indicatrices pour tester si un variant est dans le modèle
    # et nous définissons une valeur par défaut pour les poids à des fins de visualisation.
    gwas_region$in_model <- gwas_region$rsid_x %in% model$rsid

    gwas_region <- merge(gwas_region, model, by.x = "rsid_x", by.y = "rsid", all.x = T)
    min_abs_weight = min(abs(na.omit(gwas_region$weight)))
    gwas_region$weight <- ifelse(
        is.na(gwas_region$weight),
        0.8 * min_abs_weight,
        gwas_region$weight
    )
    
    list(
        model = model,
        gwas_region = gwas_region
    )
}

plot_region <- function(region, gene_name, tissue) {
    data <- get_predictdb_and_gwas_data(region, gene_name, tissue)
    
    options(repr.plot.width = 10, repr.plot.height = 4)
    options(repr.plot.res = 200)
    
    plot <- ggplot(data$gwas_region, aes(x = base_pair_location, y = neg_log_10_p_value,
                      color = in_model, size = abs(weight))) +
        geom_point(alpha = 0.7) +
        scale_color_manual(values = c("FALSE" = "#222222", "TRUE" = "#2E86C1")) +
        labs(x = paste0("Position des paires de bases (", region$chr, " ,GRCh38)"),
             y = "-log10(P)",
             color = paste0("Dans le modèle ", gene_name)) +
        theme_minimal()
    
    plot
}

In [ ]:
plot_region(wfs1_region, "WFS1", "en_Adipose_Subcutaneous")

**Interprétation du graphique de région génomique**

<div class="alert alert-info">
<p>Le graphique présente les statistiques d'association des variants génétiques situés dans la région du gène <em>WFS1</em> avec le diabète de type 2. Voici les éléments clés à analyser :</p>

<p><strong>Légende du graphique :</strong></p>
<ul>
<li><strong>Points bleus :</strong> Variants inclus dans le modèle prédisant l'expression de <em>WFS1</em> dans le tissu adipeux sous-cutané</li>
<li><strong>Taille des points :</strong> Proportionnelle au poids (importance) du variant dans le modèle prédictif</li>
<li><strong>Axe Y :</strong> Force de l'association avec le diabète de type 2 (-log10 de la valeur P)</li>
</ul>

<p><strong>Questions d'interprétation :</strong></p>
<ol>
<li>Ce graphique appuie-t-il les résultats de l'analyse TWAS ?</li>
<li>Observe-t-on une concordance entre les variants les plus fortement associés au diabète (pics les plus élevés) et ceux inclus dans le modèle prédictif (points bleus) ?</li>
<li>Comment cette visualisation peut-elle informer l'interprétation causale des résultats TWAS ?</li>
<li>Que révèle la distribution spatiale des variants du modèle par rapport au gène ?</li>
</ol>

<p><strong>Exploration supplémentaire :</strong></p>
<p>Les fonctions ci-dessous permettent de visualiser les signaux d'association de <em>WFS1</em> dans d'autres tissus. Il est recommandé de comparer les patterns observés entre les tissus et de tester également d'autres gènes significatifs identifiés dans l'analyse.</p>
</div>

In [ ]:
# plot_region(wfs1_region, "WFS1", "en_Whole_Blood")
# plot_region(wfs1_region, "WFS1", "en_Pancreas")
# plot_region(wfs1_region, "WFS1", "en_Muscle_Skeletal")